In [1]:
from importlib.util import find_spec
%load_ext autoreload
%autoreload 2

%matplotlib inline

if find_spec("qml_hep_lhc") is None:
    import sys
    sys.path.append('../..')

In [2]:
from qml_hep_lhc.data import ElectronPhoton, MNIST, QuarkGluon
from qml_hep_lhc.models import FQCNN
from tensorflow.keras.callbacks import ReduceLROnPlateau
import argparse
import numpy as np

import matplotlib.pyplot as plt
import time

2022-09-25 15:36:32.027680: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-25 15:36:32.027718: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-09-25 15:36:40.200653: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-25 15:36:40.200698: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (bhagvada): /proc/driver/nvidia/version does not exist
2022-09-25 15:36:40.201064: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
T

In [3]:
args = argparse.Namespace()

# Data
args.resize = [4,4]
args.min_max = 1
args.binary_data = [0,1]
args.batch_size = 32
args.validation_split = 0.05
args.hinge_labels = 1
args.use_quantum = 1

# Base Model
args.learning_rate = 1e-3
args.epochs = 10

# Quantum CNN Parameters
args.n_layers = 1
args.feature_map = 'AngleMap'
args.ansatz_class = 'Chen'
args.num_qconv_layers = 1
args.qconv_dims = [1]

In [4]:
data = MNIST(args)
data.prepare_data()
data.setup()
print(data)

Binarizing data...
Binarizing data...
Resizing data...
Resizing data...
Min-max scaling...
Hinge labels...
Hinge labels...

Dataset :MNIST
╒════════╤══════════════════╤════════════════╤═════════════════╤═══════════╕
│ Data   │ Train size       │ Val size       │ Test size       │ Dims      │
╞════════╪══════════════════╪════════════════╪═════════════════╪═══════════╡
│ X      │ (12031, 4, 4, 1) │ (634, 4, 4, 1) │ (2115, 4, 4, 1) │ (4, 4, 1) │
├────────┼──────────────────┼────────────────┼─────────────────┼───────────┤
│ y      │ (12031,)         │ (634,)         │ (2115,)         │ (1,)      │
╘════════╧══════════════════╧════════════════╧═════════════════╧═══════════╛

╒══════════════╤═══════╤═══════╤════════╤═══════╤══════════════════════════╕
│ Type         │   Min │   Max │   Mean │   Std │ Samples for each class   │
╞══════════════╪═══════╪═══════╪════════╪═══════╪══════════════════════════╡
│ Train Images │ -1.57 │  1.57 │  -1.35 │  0.72 │ [0, 5626]                │
├────────────

In [26]:
data.config()

{'input_dims': (4, 4, 1), 'output_dims': (1,), 'mapping': [0, 1]}

In [27]:
model = FQCNN(data.config(), args)

../../qml_hep_lhc/layers/qconv2d.py:62: UserWarning: n_qubits is only used for NQubit ansatz.
  warnings.warn('n_qubits is only used for NQubit ansatz.')


In [28]:
model.compile()
model.build_graph().summary()

Model: "FQCNN-AngleMap-Chen"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 4, 4, 1)]         0         
                                                                 
 qconv2d_0 (QConv2D)         (None, 2, 2, 1, 1)        45        
                                                                 
 flatten_2420 (Flatten)      (None, 4)                 0         
                                                                 
 TwoLayerPQC (TwoLayerPQC)   (None, 1)                 20        
                                                                 
Total params: 65
Trainable params: 52
Non-trainable params: 13
_________________________________________________________________


In [29]:
lr_scheduler_callback = ReduceLROnPlateau(monitor='val_loss',
                                              factor=np.sqrt(0.1),
                                              patience=5,
                                              min_delta=0.0001,
                                              min_lr=1e-8)
callbacks = [lr_scheduler_callback]
model.fit(data, callbacks)

Epoch 1/10
376/376 [==============================] - 128s 340ms/step - loss: 0.9881 - q_auc_2: 0.6264 - custom_accuracy: 0.5606 - val_loss: 0.9140 - val_q_auc_2: 0.7149 - val_custom_accuracy: 0.6499 - lr: 0.0010
Epoch 2/10
376/376 [==============================] - 129s 344ms/step - loss: 0.8857 - q_auc_2: 0.7134 - custom_accuracy: 0.6564 - val_loss: 0.8663 - val_q_auc_2: 0.7437 - val_custom_accuracy: 0.6851 - lr: 0.0010
Epoch 3/10
376/376 [==============================] - 125s 333ms/step - loss: 0.8570 - q_auc_2: 0.7299 - custom_accuracy: 0.6749 - val_loss: 0.8470 - val_q_auc_2: 0.7575 - val_custom_accuracy: 0.6897 - lr: 0.0010
Epoch 4/10
376/376 [==============================] - 131s 348ms/step - loss: 0.8420 - q_auc_2: 0.7445 - custom_accuracy: 0.6818 - val_loss: 0.8333 - val_q_auc_2: 0.7681 - val_custom_accuracy: 0.6900 - lr: 0.0010
Epoch 5/10
376/376 [==============================] - 134s 357ms/step - loss: 0.8292 - q_auc_2: 0.7564 - custom_accuracy: 0.6851 - val_loss: 0.8201 

In [30]:
model.test(data, callbacks=callbacks)

67/67 [==============================] - 15s 226ms/step - loss: 0.6983 - q_auc_2: 0.8269 - custom_accuracy: 0.7486


[0.6983203887939453,
 0.8269104957580566,
 0.7486007213592529,
 0.8269104957580566]

Normalize the dataset for using amplitude encoding

In [16]:
args = argparse.Namespace()

# Data
args.resize = [4,4]
args.normalize = 1
args.binary_data = [0,1]
args.batch_size = 32
args.validation_split = 0.05
args.hinge_labels = 1
args.use_quantum = 1

# Base Model
args.learning_rate = 1e-3
args.epochs = 10

# Quantum CNN Parameters
args.n_layers = 1
args.feature_map = 'AmplitudeMap'
args.ansatz = 'Chen'
args.num_qconv_layers = 1
args.qconv_dims = [1]

In [17]:
data = MNIST(args)
data.prepare_data()
data.setup()
print(data)

Binarizing data...
Binarizing data...
Resizing data...
Resizing data...
Normalizing data...
Hinge labels...
Hinge labels...

Dataset :MNIST
╒════════╤══════════════════╤════════════════╤═════════════════╤═══════════╕
│ Data   │ Train size       │ Val size       │ Test size       │ Dims      │
╞════════╪══════════════════╪════════════════╪═════════════════╪═══════════╡
│ X      │ (12031, 4, 4, 1) │ (634, 4, 4, 1) │ (2115, 4, 4, 1) │ (4, 4, 1) │
├────────┼──────────────────┼────────────────┼─────────────────┼───────────┤
│ y      │ (12031,)         │ (634,)         │ (2115,)         │ (1,)      │
╘════════╧══════════════════╧════════════════╧═════════════════╧═══════════╛

╒══════════════╤═══════╤═══════╤════════╤═══════╤══════════════════════════╕
│ Type         │   Min │   Max │   Mean │   Std │ Samples for each class   │
╞══════════════╪═══════╪═══════╪════════╪═══════╪══════════════════════════╡
│ Train Images │     0 │     1 │   0.06 │  0.21 │ [0, 5626]                │
├───────────

In [18]:
model = FQCNN(data.config(), args)
model.compile()
model.build_graph().summary()

../../qml_hep_lhc/layers/qconv2d.py:62: UserWarning: n_qubits is only used for NQubit ansatz.
  warnings.warn('n_qubits is only used for NQubit ansatz.')
../../qml_hep_lhc/encodings/amplitude.py:13: UserWarning: AmplitudeMap currently does not normalize the input unless padding is needed.
User must manually normalize the input.
  "AmplitudeMap currently does not normalize the input unless padding is needed.\nUser must manually normalize the input."
../../qml_hep_lhc/encodings/amplitude.py:13: UserWarning: AmplitudeMap currently does not normalize the input unless padding is needed.
User must manually normalize the input.
  "AmplitudeMap currently does not normalize the input unless padding is needed.\nUser must manually normalize the input."


Model: "FQCNN-AmplitudeMap-Chen"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 4, 4, 1)]         0         
                                                                 
 qconv2d_0 (QConv2D)         (None, 2, 2, 1, 1)        27        
                                                                 
 flatten_20000 (Flatten)     (None, 4)                 0         
                                                                 
 TwoLayerPQC (TwoLayerPQC)   (None, 1)                 9         
                                                                 
Total params: 36
Trainable params: 18
Non-trainable params: 18
_________________________________________________________________


In [19]:
model.fit(data, callbacks)

Epoch 1/10
376/376 [==============================] - 146s 389ms/step - loss: 1.0832 - q_auc_4: 0.4951 - custom_accuracy: 0.5273 - val_loss: 0.9960 - val_q_auc_4: 0.5000 - val_custom_accuracy: 0.5323 - lr: 0.0010
Epoch 2/10
376/376 [==============================] - 163s 434ms/step - loss: 0.9960 - q_auc_4: 0.4976 - custom_accuracy: 0.5324 - val_loss: 0.9960 - val_q_auc_4: 0.5000 - val_custom_accuracy: 0.5309 - lr: 0.0010
Epoch 3/10
376/376 [==============================] - 161s 429ms/step - loss: 0.9960 - q_auc_4: 0.4973 - custom_accuracy: 0.5324 - val_loss: 0.9960 - val_q_auc_4: 0.5000 - val_custom_accuracy: 0.5323 - lr: 0.0010
Epoch 4/10
376/376 [==============================] - 158s 421ms/step - loss: 0.9960 - q_auc_4: 0.4952 - custom_accuracy: 0.5324 - val_loss: 0.9960 - val_q_auc_4: 0.5000 - val_custom_accuracy: 0.5327 - lr: 0.0010
Epoch 5/10
376/376 [==============================] - 153s 406ms/step - loss: 0.9960 - q_auc_4: 0.4974 - custom_accuracy: 0.5324 - val_loss: 0.9960 

In [20]:
model.test(data, callbacks=callbacks)

67/67 [==============================] - 19s 282ms/step - loss: 0.9947 - q_auc_4: 0.5000 - custom_accuracy: 0.5384


[0.9947364926338196, 0.5, 0.5384017825126648, 0.5]

In [27]:
# Quantum CNN Parameters
args.n_layers = 6
args.feature_map = 'AmplitudeMap'
args.ansatz = 'Cong'
args.num_qconv_layers = 1
args.qconv_dims = [1]

In [28]:
model = FQCNN(data.config(), args)
model.compile()
model.build_graph().summary()

../../qml_hep_lhc/layers/qconv2d.py:62: UserWarning: n_qubits is only used for NQubit ansatz.
  warnings.warn('n_qubits is only used for NQubit ansatz.')
../../qml_hep_lhc/encodings/amplitude.py:13: UserWarning: AmplitudeMap currently does not normalize the input unless padding is needed.
User must manually normalize the input.
  "AmplitudeMap currently does not normalize the input unless padding is needed.\nUser must manually normalize the input."


../../qml_hep_lhc/encodings/amplitude.py:13: UserWarning: AmplitudeMap currently does not normalize the input unless padding is needed.
User must manually normalize the input.
  "AmplitudeMap currently does not normalize the input unless padding is needed.\nUser must manually normalize the input."


Model: "FQCNN-AmplitudeMap-Cong"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 4, 4, 1)]         0         
                                                                 
 qconv2d_0 (QConv2D)         (None, 2, 2, 1, 1)        57        
                                                                 
 flatten_32085 (Flatten)     (None, 4)                 0         
                                                                 
 TwoLayerPQC (TwoLayerPQC)   (None, 1)                 24        
                                                                 
Total params: 81
Trainable params: 63
Non-trainable params: 18
_________________________________________________________________


In [ ]:
model.fit(data, callbacks)

Epoch 1/10
195/376 [==============>...............] - ETA: 2:37 - loss: 0.4950 - q_auc_8: 0.9327 - custom_accuracy: 0.8726

In [ ]:
model.test(data, callbacks=callbacks)